In [11]:
import os
import pandas as pd
import numpy as np
import time

In [21]:
def manhattan_distance(a, b):
    return np.sum(np.abs(a - b), axis=1)

In [22]:
def compute_distances(point, array):
    start_time = time.time()
    distances = manhattan_distance(array, point)
    end_time = time.time()
    print(f"Time taken: {end_time - start_time} seconds")
    return distances

In [23]:
df = pd.read_csv('./route_points.csv')
a = df.values

In [24]:
point = [10.86971, 106.80231]

In [25]:
distances = compute_distances(point, a)

Time taken: 0.0 seconds


In [28]:
a[np.argmin(distances)]

array([ 10.86971143, 106.80232   ])

In [17]:
import numpy as np

# Các hằng số
a = 6378137.0  # Bán kính Trái Đất theo đường xích đạo (m)
f = 1 / 298.257223563  # Độ dẹt của Trái Đất
e2 = 2 * f - f ** 2  # Độ lệch tâm bình phương

def lla_to_ecef(lat, lon, alt):
    lat = np.deg2rad(lat)
    lon = np.deg2rad(lon)

    N = a / np.sqrt(1 - e2 * np.sin(lat) ** 2)

    X = (N + alt) * np.cos(lat) * np.cos(lon)
    Y = (N + alt) * np.cos(lat) * np.sin(lon)
    Z = (N * (1 - e2) + alt) * np.sin(lat)

    return np.array([X, Y, Z])

def ecef_to_ned(ecef, ref_lla):
    ref_ecef = lla_to_ecef(*ref_lla)
    dX = ecef - ref_ecef

    lat, lon = np.deg2rad(ref_lla[0]), np.deg2rad(ref_lla[1])
    R = np.array([[-np.sin(lat) * np.cos(lon), -np.sin(lat) * np.sin(lon), np.cos(lat)],
                  [-np.sin(lon), np.cos(lon), 0],
                  [-np.cos(lat) * np.cos(lon), -np.cos(lat) * np.sin(lon), -np.sin(lat)]])

    ned = R @ dX
    return ned

def ned_to_ecef(ned, ref_lla):
    ref_ecef = lla_to_ecef(*ref_lla)
    lat, lon = np.deg2rad(ref_lla[0]), np.deg2rad(ref_lla[1])

    R = np.array([[-np.sin(lat) * np.cos(lon), -np.sin(lat) * np.sin(lon), np.cos(lat)],
                  [-np.sin(lon), np.cos(lon), 0],
                  [-np.cos(lat) * np.cos(lon), -np.cos(lat) * np.sin(lon), -np.sin(lat)]])

    dX = np.linalg.inv(R) @ ned
    ecef = ref_ecef + dX

    return ecef

def ecef_to_lla(ecef):
    X, Y, Z = ecef
    lon = np.arctan2(Y, X)

    p = np.sqrt(X ** 2 + Y ** 2)
    lat = np.arctan2(Z, p * (1 - e2))
    alt = 0

    for _ in range(5):
        N = a / np.sqrt(1 - e2 * np.sin(lat) ** 2)
        alt = p / np.cos(lat) - N
        lat = np.arctan2(Z, p * (1 - e2 * N / (N + alt)))

    lat = np.rad2deg(lat)
    lon = np.rad2deg(lon)

    return np.array([lat, lon, alt])

def lla_to_ned(lla, ref_lla):
    ecef = lla_to_ecef(*lla)
    ned = ecef_to_ned(ecef, ref_lla)
    return ned

def ned_to_lla(ned, ref_lla):
    ecef = ned_to_ecef(ned, ref_lla)
    lla = ecef_to_lla(ecef)
    return lla

In [21]:
# Ví dụ sử dụng

# Tọa độ gốc tham chiếu (Latitude, Longitude, Altitude)
ref_lla = [10.87050, 106.802, 0.0]  # Một vị trí ở TP.HCM, Việt Nam

# Tọa độ cần chuyển đổi (Latitude, Longitude, Altitude)
target_lla = [10.87043, 106.80196, 0.0]  # Thêm 10m cao độ

# Chuyển từ LLA sang NED
ned = lla_to_ned(target_lla, ref_lla)
print("NED:", ned)

ned[2] = 0

# Chuyển ngược từ NED sang LLA
lla = ned_to_lla(ned, ref_lla)
print("LLA:", lla)

NED: [-7.74296416e+00 -4.37340039e+00  6.22851312e-06]
LLA: [1.08704300e+01 1.06801960e+02 6.22775406e-06]
